## Coursera - IBM Data Science Capstone Project ##

### Brian Lee ###
BrianTrewLee@gmail.com<br />
LinkedIn: https://www.linkedin.com/in/brian-lee-64a2022b/



## 1. Introduction ##

If I were to open a Starbucks, where should I locate it? Clearly, the location is one of the most important business decisions for this venture. Is there a science to locating a StarBucks? Can we create a machine learning model to predict where a StarBucks is likely to be located? I'd like to find out.

If a reliable model can be made,it could be used in the process of opening a store. It could be used as a final sanity check, or at the beginning stage, to select a promising location. If a trustworthy model predicts with a high degree of confidence that a Starbucks should be located in an area, but there is not one there yet, perhaps that is an opportunity.

Any person interested in opening a Starbucks should be interested in these results. This includes myself, other potential franchisees, and the Starbucks corporation itself, which operates many of it's own stores. The Starbucks competition might also be interested as they could possibly gain competitive insigths. I also believe others might be interested in this procedure, as it might be applied to predicting the location of other entities, based on the same sort of data.

The main purpose of this project is to prove the concept of predicting Starbuck's locations in general. I suspect it may work better or worse depending on the locations chosen for training and prediction, the radius size, and the specific features used in the model. I may vary those factors in order to prove the concept, which could then be applied carefully to a particular geographic area of interest at a later date.

### 2. Data ###

Foursquare (https://foursquare.com) is a company that crowdsources location data, tying latitude-longitude coordinates (and other things) with public venues, including many businesses, such as Starbucks locations. The features for the Starbucks location classifier will come from venue data from the Fourquare API. I will use venue category names (such as 'bar', 'Chinese restaurant', 'coffee shop') as features to classify an area as either an area with a Starbucks, or an area without one. I will use the venue name (e.g. 'Starbucks') to determine whether or not the venue is a Starbucks. An area containing a Starbucks contains at least one venue named 'Starbucks' within the radius supplied to the Foursquare API's explore' endpoint.

An area will be a circular area with a given radius. I have not determined that radius yet, and I might use more than one radius in this project. Within that radius, the Foursquare API will return venues. The larger the radius, the more venues that could be anticipated, and the higher likelihood of a Starbucks, but a larger radius is also less useful for business use. I have done some exploratory work with a radius of 300m.

The areas to be chosen should contain a mix of areas with Starbuck's and those without. I have done some exploratory work in a particular area of San Francisco. I located Starbucks with Google Maps and got latitude and longitude points making a box around that area. I could choose the train and test points at random within an area, or I could generate a grid covering an entire area. Given the constraints my Foursquare Developer account places on my searches, and knowing that binary classifiers work better with a good number of examples of both classes, I may want to generate areas that I know will be heavy with Starbucks locations. To that end, I discovered a Kaggle dataset (https://www.kaggle.com/starbucks/store-locations) containing the latitude and longitude coordinates of 25,600 worldwide Starbucks locations. I may or may not use that resource.

### 3. Methodology ###

I will generate a corpus of geographic points, and using a reasonable radius (perhaps 300m) a corpus of geographic areas. Each geographic area will have a feature set based on json data returned by the Foursquare API's 'explore' endpoint. I plan to use the category names of the returned venues as features for that area. I will use the venue names to determine whether oir not a Starbucks is in that area, and generate the labels from that. 

Once I have labelled data, I will use sklearn.model_selection.train_test_split to generate tarining and testing sets. Then I  can use any of a host of sklearn classifiers to generate predictive models. I have done some exploratory work with a random forest.

In [2]:
# import libraries

import urllib.request # open and read URLs

import json # handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # handle requests
import pandas as pd # process data as dataframes with Pandas
import numpy as np # handle data in a vectorized manner with NumPy

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't installed the GeoPy geocoding library yet
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

##!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't installed the Folium library yet
import folium # map rendering library

# Matplolib plotting library and associated modules
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # for K-Means clustering with Scikit-Learn

print("Libraries imported!")

Libraries imported!


In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
# Define a function to gather recommended venues, with specifically the name and category, using the explore API

def getNearbyVenues(latitudes, longitudes, radius=300, limit=100):
    
    count = 0
    venues_list = []
    for lat, lng in zip(latitudes, longitudes):
        #create a unique namefor this pointthat is easier todeal with than a combination of lat and lon
        point_name = "point"+str(count)
        print(point_name,':',lat,'-',lng)
         

            
        # create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        #print("resultscols=",results.columns)
        

        
        # return only relevant information for each nearby venue
        venues_list.append([(
            point_name,
            lat, 
            lng, 
            v["venue"]["name"], 
            v["venue"]["location"]["lat"], 
            v["venue"]["location"]["lng"],  
            v["venue"]["location"]["distance"], 
            v["venue"]["categories"][0]["name"]) for v in results])
        count = count + 1
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Point Name",
                  "Neighborhood Latitude", 
                  "Neighborhood Longitude", 
                  "Venue", 
                  "Venue Latitude", 
                  "Venue Longitude", 
                  "Venue Distance",           
                  "Venue Category"]
    
    return(nearby_venues)

In [5]:
sf_lat=37.7749
sf_lon=-122.4194
ch_lat=41.8781
ch_lon=-87.6298
la_lat=34.0522
la_lon=-118.2437
starbucks_m_lat=37.744644
starbucks_m_lon=-122.452731
import random

#37.780253, -122.509117
#37.804203, -122.459264 - upper left of starbucks intensive space
#37.767547, -122.409342 - lower right ofstarbucksintensive space
llat=37.767547
ulat=37.804203
llon=-122.459264
ulon=-122.409342
random_lat0 = random.uniform(37.767547,37.804203)
random_lon0 = random.uniform(-122.459264, -122.409342)
random_lat1 = random.uniform(llat,ulat)
random_lon1 = random.uniform(llon, ulon)
random_lat2 = random.uniform(llat,ulat)
random_lon2 = random.uniform(llon, ulon)
random_lat3 = random.uniform(llat,ulat)
random_lon3 = random.uniform(llon, ulon)
random_lat4 = random.uniform(llat,ulat)
random_lon4 = random.uniform(llon, ulon)
random_lat5 = random.uniform(llat,ulat)
random_lon5 = random.uniform(llon, ulon)
random_lat6 = random.uniform(llat,ulat)
random_lon6 = random.uniform(llon, ulon)
random_lat7 = random.uniform(llat,ulat)
random_lon7 = random.uniform(llon, ulon)
random_lat8 = random.uniform(llat,ulat)
random_lon8 = random.uniform(llon, ulon)
random_lat9 = random.uniform(llat,ulat)
random_lon9 = random.uniform(llon, ulon)

data = pd.DataFrame(np.array([[sf_lat, sf_lon], [ch_lat, ch_lon], [la_lat, la_lon], [starbucks_m_lat,starbucks_m_lon], [random_lat0,random_lon0], [random_lat1,random_lon1], [random_lat2,random_lon2], [random_lat3,random_lon3], [random_lat4,random_lon4], [random_lat5,random_lon5], [random_lat6,random_lon6], [random_lat7,random_lon7], [random_lat8,random_lon8], [random_lat9,random_lon9] ]), columns=['Latitude', 'Longitude'])

data



,Latitude,Longitude
0,37.774900,-122.419400
1,41.878100,-87.629800
2,34.052200,-118.243700
3,37.744644,-122.452731
4,37.784163,-122.449484
5,37.785669,-122.453940
6,37.801934,-122.456720
7,37.780781,-122.429497
8,37.783958,-122.411473
9,37.785866,-122.449851


In [6]:
random_latlons = []
for j in range(0,250):
    random_lat = random.uniform(llat,ulat)
    random_lon = random.uniform(llon,ulon)
    random_latlon=(random_lat,random_lon)
    random_latlons.append(random_latlon)
    

In [7]:
random_latlons = pd.DataFrame(random_latlons)
random_lats=random_latlons[0]
random_lons=random_latlons[1]
random_lats.shape



(250,)

In [8]:
random_latlons[0].shape

(250,)

In [9]:
random_latlons[1].shape

(250,)

In [10]:


data["Latitude"] = random_latlons[0]
data["Longitude"] = random_latlons[1]

data["Latitude"].head()
data["Latitude"].shape


(14,)

In [11]:
#37.599350, -122.496830
#37.807957, -122.390854
#reset parameters to bigger area
##llat = 37.599350
#ulat = 37.807957
##llon = -122.496830
##ulon = -122.390854


points = []
radius = 1000 #in meters
current_lat = ulat
current_lon = llon
import geopy
from geopy.distance import VincentyDistance
d = radius/1000 #in kilometers
#d=d*3
b=90 #in degrees 90 = east,180=south,270=west
# given: lat1, lon1, b = bearing in degrees, d = distance in kilometers

def lon_pass(current_lat, current_lon, ulat, ulon):
    while (current_lat <= ulat) and (current_lon <= ulon) and (current_lat >= llat) and (current_lon >= llon):
        origin = geopy.Point(current_lat, current_lon)
        destination = VincentyDistance(kilometers=d).destination(origin, b)

        current_lat, current_lon = destination.latitude, destination.longitude
        print(current_lat, current_lon)
        points.append((current_lat, current_lon))

while (current_lat <= ulat) and (current_lon <= ulon) and (current_lat >= llat) and (current_lon >= llon):
    print('first line of while')
    lon_pass(current_lat, current_lon, ulat, ulon)
    print('new row of coordinates')  
    current_lon=llon#move back to beginning of row   
    origin = geopy.Point(current_lat, current_lon)
    destination = VincentyDistance(kilometers=d).destination(origin, 180)#move south one step for next row
    current_lat, current_lon = destination.latitude, destination.longitude

    
    


first line of while
37.80414827546323 -122.34571221931947
new row of coordinates


We many need more examples of the Starbucks class for a balanced set - so get a bunch of points we know are near to Starbucks

I pulled coordinates from the Kaggle Starbucks locations dataset

In [12]:
#hardcoded coordinates near Starbucks 
points.append((34.51, -117.4))
points.append((34.16, -118.76))
points.append((33.87, -117.75))
points.append((37.76, -122.25))
points.append((37.85, -122.03))
points.append((37.88, -122.31))
points.append((34.09, -118.13))
points.append((33.59, -117.72))
points.append((32.83, -116.75))
points.append((38.18, -122.25))
points.append((33.80, -117.92))
points.append((33.86, -117.74))
points.append((40.43, -122.29))
points.append((38.08, -120.56))
points.append((38.71, -121.37))
points.append((37.96, -121.74))
points.append((34.52, -117.22))
points.append((36.98, -121.91))
points.append((34.14, -118.05))
points.append((35.13, -120.6))
points.append((33.87, -118.08))
points.append((34.99, -118.94))
points.append((35.49, -120.66))
points.append((37.34, -120.59))

In [13]:
starbucks_heavy_coordinates = pd.read_csv('data/directory-us-ca.csv')


In [14]:
starbucks_heavy_coordinates

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Longitude,Latitude
0,Starbucks,10429-100710,Palmdale & Hwy 395,Company Owned,14136 US Hwy 395,Adelanto,CA,US,923016700,-117.40,34.51
1,Starbucks,635-352,Kanan & Thousand Oaks,Company Owned,5827 Kanan Road,Agoura,CA,US,913011651,-118.76,34.16
2,Starbucks,74510-27669,Vons-Agoura Hills #2001,Licensed,5671 Kanan Rd.,Agoura Hills,CA,US,913013358,-118.76,34.15
3,Starbucks,29839-255026,Target Anaheim T-0677,Licensed,8148 E SANTA ANA CANYON ROAD,AHAHEIM,CA,US,92808,-117.75,33.87
4,Starbucks,23463-230284,Safeway - Alameda 3281,Licensed,2600 5th Street,Alameda,CA,US,94501,-122.28,37.79
5,Starbucks,6479-62999,Park & Central Alameda,Company Owned,1364 Park Street,Alameda,CA,US,945014542,-122.24,37.76
6,Starbucks,5535-728,Webster & Atlantic - Alameda,Company Owned,720 Atlantic Avenue,Alameda,CA,US,945012176,-122.28,37.78
7,Starbucks,74877-100291,Safeway - Alameda #2708,Licensed,2227 South Shore Center,Alameda,CA,US,945018029,-122.25,37.76
8,Starbucks,11161-103516,Tilden & Blanding,Company Owned,"2671 Blanding Avene, D",Alameda,CA,US,945011587,-122.23,37.77
9,Starbucks,19859-196187,Target Alameda T-2829,Licensed,2700 Fifth St,Alameda,CA,US,94501,-122.28,37.78


In [15]:
num_needed_to_balance = len(points)
num_needed_to_balance

2000

In [16]:
starbucks_heavy_coordinates['Latitude'][1]  

34.16

In [17]:
shc_lat, shc_lon = starbucks_heavy_coordinates.Latitude, starbucks_heavy_coordinates.Longitude


#nl = pd.DataFrame(list((shc_lat[0:num_needed_to_balance],shc_lon[0:num_needed_to_balance]))).transpose
#points.append(nl)

nnl=(shc_lat[0:num_needed_to_balance],shc_lon[0:num_needed_to_balance])
type(nnl)
pnnl = pd.DataFrame(nnl)
pnnl.shape
pnnl = pnnl.transpose()
#pnnl = pnnl.rename(index=str, columns={"Latitude": "0", "Longitude": "1"})
pnnl
newlats=pnnl['Latitude']
newlons=pnnl['Longitude']
newlats
#points.append(list((newlats,newlons)))
points = pd.DataFrame(points)
###shc_latlons = []
current_end = len(points)
for jjj in range(0,num_needed_to_balance):
    #points[0][(current_end+jjj)]=newlats[jjj]
    #points.append(list(newlats[jjj],newlons[jjj])
    ###new_balancing_point_lat = starbucks_heavy_coordinates['Latitude'][jjj]    
    ###new_balancing_point_lon = starbucks_heavy_coordinates['Longitude'][jjj]
    ###print(str(jjj) + ':' + str(new_balancing_point_lat) + '-' + str(new_balancing_point_lon))   
    ###points.append(list((new_balancing_point_lat, new_balancing_point_lon)))
    print(str(jjj))
    lenp = len(points)
    points = points.set_value(lenp, 0, newlats[jjj])
    points = points.set_value(lenp, 1, newlons[jjj])

points    
#points.append((37.34, -120.59))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422


,0,1
0,37.804148,-122.345712
1,34.510000,-117.400000
2,34.160000,-118.760000
3,33.870000,-117.750000
4,37.760000,-122.250000
5,37.850000,-122.030000
6,37.880000,-122.310000
7,34.090000,-118.130000
8,33.590000,-117.720000
9,32.830000,-116.750000


In [18]:
points.shape

(2025, 2)

shuffle the points

In [19]:
points = points.sample(frac=1).reset_index(drop=True)
points.head()

,0,1
0,34.14,-117.86
1,34.15,-118.12
2,36.74,-119.79
3,33.07,-117.07
4,33.89,-117.87


In [20]:
points.tail()

,0,1
2020,34.06,-118.36
2021,37.71,-120.96
2022,33.56,-117.65
2023,38.66,-121.31
2024,33.92,-118.10


In [21]:
points

,0,1
0,34.14,-117.86
1,34.15,-118.12
2,36.74,-119.79
3,33.07,-117.07
4,33.89,-117.87
5,34.21,-118.22
6,38.58,-121.28
7,33.96,-117.59
8,36.54,-121.91
9,34.00,-117.73


In [22]:
map = folium.Map(location=[ulat, ulon], zoom_start=10)


# add markers to map
for lat, lng in zip(points[0],points[1]):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
map

In [23]:
#getrid#data["Longitude"].head()
#getrid#data.shape

In [24]:

#venues = getNearbyVenues(latitudes = data["Latitude"],
#                               longitudes = data["Longitude"]
#                               )
venues = getNearbyVenues(latitudes = points[0],
                              longitudes =  points[1]
                               )


point0 : 34.14 - -117.86
point1 : 34.15 - -118.12
point2 : 36.74 - -119.79
point3 : 33.07 - -117.07
point4 : 33.89 - -117.87
point5 : 34.21 - -118.22
point6 : 38.58 - -121.28
point7 : 33.96 - -117.59
point8 : 36.54 - -121.91
point9 : 34.0 - -117.73
point10 : 33.82 - -116.51
point11 : 35.12 - -120.6
point12 : 36.79 - -121.67
point13 : 34.13 - -118.26
point14 : 33.56 - -117.63
point15 : 33.69 - -117.34
point16 : 34.09 - -117.67
point17 : 34.05 - -118.47
point18 : 32.85 - -117.27
point19 : 33.1 - -117.31
point20 : 33.79 - -117.87
point21 : 33.58 - -117.63
point22 : 37.64 - -118.97
point23 : 33.87 - -118.08
point24 : 32.73 - -117.2
point25 : 33.61 - -117.68
point26 : 33.66 - -117.3
point27 : 38.56 - -121.49
point28 : 33.72 - -118.04
point29 : 32.77 - -115.53
point30 : 40.59 - -122.35
point31 : 33.72 - -116.2
point32 : 32.92 - -117.12
point33 : 37.81 - -121.99
point34 : 33.86 - -118.09
point35 : 33.92 - -117.33
point36 : 37.98 - -122.56
point37 : 33.68 - -117.91
point38 : 38.73 - -121.29
po

KeyError: 'groups'

In [ ]:
venues.tail()

In [ ]:
venues['Venue'].unique()
venues['Venue'].value_counts()

In [ ]:
venues[350:400]


# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['point_name'] = venues['Point Name'] 
onehot['point_lat'] = venues['Neighborhood Latitude'] 
onehot['point_lon'] = venues['Neighborhood Longitude'] 
onehot['isStarbucks'] = (venues['Venue'] == "Starbucks")
# move neighborhood column to the first column
fixed_columns = [onehot.columns[-3]] + [onehot.columns[-2]] + [onehot.columns[-1]] + list(onehot.columns[:-3])
onehot = onehot[fixed_columns]

onehot.isStarbucks = onehot.isStarbucks.astype(int)

onehot.head()
onehot['isStarbucks'].value_counts()

In [ ]:
venues.head()

In [ ]:
grouped = onehot.groupby('point_name').mean().reset_index()
grouped

In [ ]:
num_top_venues = 5

for point in grouped['point_name']:
    print("----"+point+"----")
    temp = grouped[grouped['point_name'] == point].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
grouped.head()

In [ ]:
y = grouped['isStarbucks'] > 0
y = pd.DataFrame(y)
y = y.isStarbucks.astype(int)
print(y.value_counts())
X = grouped
print(X.shape)
X .drop(columns=['point_name', 'point_lat', 'point_lon', 'isStarbucks'], inplace=True)
print(X.columns)
print(X.shape)

In [ ]:
from sklearn.model_selection import train_test_split
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
pdytrain=pd.DataFrame(y_train)
pdytest=pd.DataFrame(y_test)

pdytrain['isStarbucks'].value_counts()


In [ ]:
pdytest['isStarbucks'].value_counts()

In [ ]:
import sklearn as sk  
from sklearn.ensemble import RandomForestClassifier

#RF = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)  
RF = RandomForestClassifier(n_estimators=100, max_depth=200, random_state=0)  
RF.fit(X_train, y_train)  
RF.predict(X_train)  
RF.score(X_train,y_train)

In [ ]:
print('test')

In [ ]:
ybogus = y_test-1


In [ ]:
pred_test=RF.predict(X_test)  
RF.score(X_test,y_test)

In [ ]:
# What about AUROC?
prob_y = RF.predict_proba(X)
prob_y
prob_y_train = RF.predict_proba(X_train)
prob_y_test = RF.predict_proba(X_test)


In [ ]:
#prob_y = [p[1] for p in prob_y]
#prob_y
#print( roc_auc_score(y, prob_y) )

In [ ]:
#y_test
from sklearn.metrics import roc_curve, roc_auc_score
print(roc_auc_score(y_train, prob_y_train[:,1]))
print(roc_auc_score(y_test, prob_y_test[:,1]))


In [ ]:
estimator = RF.estimators_[0]

In [ ]:
estimator

In [ ]:
from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = X.columns,
                class_names = 'has_starbucks',
                rounded = True, proportion = False, 
                precision = 2, filled = True)

In [ ]:
# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')